# 01_Notebook_Eksplorasi: Deteksi Kantuk dengan MediaPipe Face Mesh

Proyek ini bertujuan untuk mengembangkan sistem deteksi kantuk real-time menggunakan pustaka MediaPipe Face Mesh. Sistem ini menganalisis dua indikator utama:

1.  **Eye Aspect Ratio (EAR)**: Mengukur seberapa tertutup mata seseorang. Nilai EAR yang rendah secara konsisten menunjukkan mata tertutup dan kemungkinan kantuk.
2.  **Mouth Aspect Ratio (MAR)**: Mengukur seberapa terbuka mulut seseorang, yang dapat mengindikasikan menguap. Nilai MAR yang tinggi secara konsisten menunjukkan menguap dan kemungkinan kantuk.

Model ini akan memberikan peringatan visual dan suara jika terdeteksi kantuk berdasarkan ambang batas yang dapat dikonfigurasi.

---


In [ ]:
## Persiapan Lingkungan dan Instalasi Pustaka (Untuk Uji Offline)

# Instalasi pustaka yang diperlukan
# Kita tidak membutuhkan streamlit dan streamlit-webrtc untuk pengujian offline
!pip install mediapipe==0.10.21 opencv-python==4.10.0.84 opencv-contrib-python==4.11.0.86 playsound==1.3.0 av==14.4.0 numpy==1.26.4 scipy==1.13.1 protobuf==4.25.8 pyopenssl==25.1.0 cryptography==45.0.4 ffmpeg-python==0.2.0

# PENTING: playsound mungkin tidak berfungsi di lingkungan Colab karena tidak ada output audio langsung.
print("\nInstalasi pustaka selesai.")


Instalasi pustaka selesai.


In [ ]:
## Implementasi Model Deteksi Kantuk (Modifikasi untuk Uji Offline)

import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist
import av # Masih perlu untuk VideoFileClip
from playsound import playsound
import threading
import os
from google.colab.patches import cv2_imshow # Untuk menampilkan gambar di Colab

# --- Konstanta untuk Mata ---
DEFAULT_EAR_THRESHOLD = 0.20
DEFAULT_CONSECUTIVE_FRAMES_EYE = 20

# --- Konstanta untuk Mulut (Menguap) ---
DEFAULT_MAR_THRESHOLD = 0.6
DEFAULT_CONSECUTIVE_FRAMES_YAWN = 25

# --- Pastikan file alarm ada ---
ALARM_SOUND_PATH = "alarm.mp3"
if not os.path.exists(ALARM_SOUND_PATH):
    print(f"ERROR: File alarm '{ALARM_SOUND_PATH}' tidak ditemukan. Pastikan file tersebut ada di folder yang sama.")
    # Exit atau tangani error sesuai kebutuhan jika file tidak ada
    # Untuk uji offline, kita bisa lanjutkan tanpa suara jika file tidak ada
    ALARM_SOUND_PATH = None # Set ke None agar tidak mencoba memutar

# --- Inisialisasi MediaPipe ---
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# Landmark indices untuk mata
LEFT_EYE_INDICES = [362, 385, 387, 263, 373, 380]
RIGHT_EYE_INDICES = [33, 160, 158, 133, 153, 144]

# Landmark indices untuk mulut
MOUTH_VERTICAL_INDICES = [13, 14]
MOUTH_HORIZONTAL_INDICES = [78, 308]

def calculate_ear(eye_landmarks):
    v1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    v2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    h1 = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    if h1 == 0:
        return 0
    ear = (v1 + v2) / (2.0 * h1)
    return ear

def calculate_mar(mouth_landmarks):
    v_dist = dist.euclidean(mouth_landmarks[0], mouth_landmarks[1])
    h_dist = dist.euclidean(mouth_landmarks[2], mouth_landmarks[3])
    if h_dist == 0:
        return 0
    mar = v_dist / h_dist
    return mar

# --- Kelas Prosesor Deteksi (Disesuaikan dari Streamlit-webrtc) ---
# Kita tidak lagi mewarisi VideoProcessorBase karena tidak menggunakan streamlit-webrtc
class DrowsinessDetectorStandalone:
    def __init__(self, ear_threshold, consecutive_frames_eye, mar_threshold, consecutive_frames_yawn):
        self.ear_threshold = ear_threshold
        self.consecutive_frames_eye = consecutive_frames_eye
        self.mar_threshold = mar_threshold
        self.consecutive_frames_yawn = consecutive_frames_yawn

        self.eye_frame_counter = 0
        self.yawn_frame_counter = 0
        self.drowsy_alert_triggered = False # Tetap ada untuk logika alarm
        self.face_mesh = mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5
        )

    def process_frame(self, image):
        """Memproses satu frame gambar dan mengembalikan frame yang sudah di-annotate."""
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        rgb_image.flags.writeable = False
        results = self.face_mesh.process(rgb_image)
        rgb_image.flags.writeable = True

        alert_text = ""
        is_drowsy = False

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)

                landmarks = np.array([(lm.x * image.shape[1], lm.y * image.shape[0]) for lm in face_landmarks.landmark], dtype=np.int32)

                left_eye_lm = landmarks[LEFT_EYE_INDICES]
                right_eye_lm = landmarks[RIGHT_EYE_INDICES]
                left_ear = calculate_ear(left_eye_lm)
                right_ear = calculate_ear(right_eye_lm)
                avg_ear = (left_ear + right_ear) / 2.0
                cv2.putText(image, f"EAR: {avg_ear:.2f}", (image.shape[1] - 150, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                mouth_v_lm = landmarks[MOUTH_VERTICAL_INDICES]
                mouth_h_lm = landmarks[MOUTH_HORIZONTAL_INDICES]
                mouth_lm = np.concatenate((mouth_v_lm, mouth_h_lm))
                mar = calculate_mar(mouth_lm)
                cv2.putText(image, f"MAR: {mar:.2f}", (image.shape[1] - 150, 60),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

                if avg_ear < self.ear_threshold:
                    self.eye_frame_counter += 1
                else:
                    self.eye_frame_counter = 0

                if mar > self.mar_threshold:
                    self.yawn_frame_counter += 1
                else:
                    self.yawn_frame_counter = 0

                if self.eye_frame_counter >= self.consecutive_frames_eye:
                    is_drowsy = True
                    alert_text = "MATA MENGANTUK!"

                if self.yawn_frame_counter >= self.consecutive_frames_yawn:
                    is_drowsy = True
                    alert_text = "MENGUAP TERDETEKSI!" if alert_text == "" else alert_text + " & MENGUAP"

                if is_drowsy:
                    cv2.putText(image, alert_text, (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                    if not self.drowsy_alert_triggered: # Tidak perlu cek ALARM_SOUND_PATH di sini
                        self.drowsy_alert_triggered = True
                        if ALARM_SOUND_PATH: # Beri tahu bahwa alarm *seharusnya* berbunyi
                            print("Peringatan: Alarm kantuk/menguap terpicu! (Suara tidak didukung di Colab)")
                        else:
                            print("Peringatan: Alarm kantuk/menguap terpicu! (File alarm tidak ditemukan)")

                        # Baris ini yang dihapus/dikomentari untuk menghindari error playsound di Colab
                        # try:
                        #     threading.Thread(target=playsound, args=(ALARM_SOUND_PATH,)).start()
                        # except Exception as e:
                        #     print(f"Error playing sound: {e} (Mungkin tidak didukung di Colab)")
                else:
                    self.drowsy_alert_triggered = False

        return image, is_drowsy # Mengembalikan gambar yang sudah diproses dan status kantuk

# --- Fungsi untuk Menguji dari Video File ---
def test_from_video(video_path, detector, display_frames=True, max_frames=None):
    if not os.path.exists(video_path):
        print(f"ERROR: Video file '{video_path}' tidak ditemukan.")
        return

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"ERROR: Tidak dapat membuka video file '{video_path}'.")
        return

    frame_count = 0
    drowsy_frames = 0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Memproses video: {video_path} ({total_frames} frames)")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        processed_frame, is_drowsy = detector.process_frame(frame)

        if is_drowsy:
            drowsy_frames += 1

        if display_frames and frame_count % 30 == 0: # Tampilkan setiap 30 frame untuk kecepatan
             print(f"Frame {frame_count}/{total_frames} - Drowsy: {is_drowsy}")
             cv2_imshow(processed_frame)
             # Tambahkan delay singkat agar gambar dapat terlihat
             # time.sleep(0.01) # Jika ingin menambahkan delay, import time

        if max_frames and frame_count >= max_frames:
            break

    cap.release()
    print(f"\n--- Ringkasan Video '{video_path}' ---")
    print(f"Total frame diproses: {frame_count}")
    print(f"Total frame terdeteksi kantuk/menguap: {drowsy_frames}")
    print(f"Persentase kantuk: {drowsy_frames / frame_count * 100:.2f}%")

# --- Fungsi untuk Menguji dari Gambar File ---
def test_from_image(image_path, detector):
    if not os.path.exists(image_path):
        print(f"ERROR: Gambar file '{image_path}' tidak ditemukan.")
        return

    image = cv2.imread(image_path)
    if image is None:
        print(f"ERROR: Tidak dapat membaca gambar file '{image_path}'.")
        return

    processed_image, is_drowsy = detector.process_frame(image)
    print(f"\n--- Ringkasan Gambar '{image_path}' ---")
    print(f"Terdeteksi kantuk/menguap: {is_drowsy}")
    cv2_imshow(processed_image)

print("Fungsi deteksi dan pengujian telah dimuat.")

Fungsi deteksi dan pengujian telah dimuat.


In [ ]:
## Bagian Uji Model

from google.colab import files
import os # Pastikan modul os diimpor untuk os.path.exists

# Inisialisasi Detektor dengan parameter default (bisa Anda ubah di sini)
detector = DrowsinessDetectorStandalone(
    ear_threshold=DEFAULT_EAR_THRESHOLD,
    consecutive_frames_eye=DEFAULT_CONSECUTIVE_FRAMES_EYE,
    mar_threshold=DEFAULT_MAR_THRESHOLD,
    consecutive_frames_yawn=DEFAULT_CONSECUTIVE_FRAMES_YAWN
)

print("Detektor siap.")

# --- Bagian Uji Gambar (Dinonaktifkan agar tidak membingungkan) ---
# Contoh: Uji gambar 'ngantuk.jpg' yang sudah ada di root direktori sesi Colab
# print("\n--- Uji Gambar: ngantuk.jpg ---")
# try:
#     image_path_ngantuk = "ngantuk.jpg" # Bisa juga ditulis "/content/ngantuk.jpg"
#     if not os.path.exists(image_path_ngantuk):
#         print(f"File '{image_path_ngantuk}' tidak ditemukan. Harap pastikan nama file dan path sudah benar.")
#     else:
#         test_from_image(image_path_ngantuk, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar ngantuk.jpg: {e}")

# --- Bagian Uji Gambar Lainnya (Dinonaktifkan agar tidak membingungkan) ---
# Contoh 1: Uji gambar mata terbuka (dinonaktifkan)
# print("\n--- Uji Gambar: Mata Terbuka ---")
# try:
#     image_path_open_eyes = "/content/wajah_terbuka.jpg" # GANTI DENGAN PATH GAMBAR ANDA
#     if not os.path.exists(image_path_open_eyes):
#         print(f"File '{image_path_open_eyes}' tidak ditemukan. Harap unggah terlebih dahulu.")
#     else:
#         test_from_image(image_path_open_eyes, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar terbuka: {e}")

# Contoh 2: Uji gambar mata tertutup (dinonaktifkan)
# print("\n--- Uji Gambar: Mata Tertutup ---")
# try:
#     image_path_closed_eyes = "/content/wajah_tertutup.jpg" # GANTI DENGAN PATH GAMBAR ANDA
#     if not os.path.exists(image_path_closed_eyes):
#         print(f"File '{image_path_closed_eyes}' tidak ditemukan. Harap unggah terlebih dahulu.")
#     else:
#         test_from_image(image_path_closed_eyes, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar tertutup: {e}")

# --- Contoh Penggunaan: Uji Video (Aktifkan ini) ---
# Unggah file video Anda ke Colab dan ganti nama file di bawah
print("\n--- Uji Video: ngantuk.mp4 ---")
try:
    video_path_test = "ngantuk.mp4" # Path ke video Anda yang sudah diunggah
    # Anda juga bisa menggunakan "/content/ngantuk.mp4"

    if not os.path.exists(video_path_test):
        print(f"File '{video_path_test}' tidak ditemukan. Harap pastikan nama file dan path sudah benar.")
    else:
        # Untuk video, set display_frames=True jika ingin melihat frame satu per satu (akan menghasilkan banyak output dan mungkin lambat)
        # Atau False untuk hanya melihat ringkasan teks.
        # max_frames bisa dibatasi untuk video panjang agar tidak terlalu lama.
        test_from_video(video_path_test, detector, display_frames=False, max_frames=None) # Set max_frames=None untuk proses semua frame
except Exception as e:
    print(f"Error saat menguji video: {e}")

Detektor siap.

--- Uji Video: ngantuk.mp4 ---
Memproses video: ngantuk.mp4 (301 frames)
Peringatan: Alarm kantuk/menguap terpicu! (Suara tidak didukung di Colab)

--- Ringkasan Video 'ngantuk.mp4' ---
Total frame diproses: 301
Total frame terdeteksi kantuk/menguap: 282
Persentase kantuk: 93.69%


In [ ]:
## Bagian Uji Model

from google.colab import files
import os # Pastikan modul os diimpor untuk os.path.exists

# Inisialisasi Detektor dengan parameter default (bisa Anda ubah di sini)
detector = DrowsinessDetectorStandalone(
    ear_threshold=DEFAULT_EAR_THRESHOLD,
    consecutive_frames_eye=DEFAULT_CONSECUTIVE_FRAMES_EYE,
    mar_threshold=DEFAULT_MAR_THRESHOLD,
    consecutive_frames_yawn=DEFAULT_CONSECUTIVE_FRAMES_YAWN
)

print("Detektor siap.")

# --- Bagian Uji Gambar (Dinonaktifkan agar tidak membingungkan) ---
# Contoh: Uji gambar 'ngantuk.jpg' yang sudah ada di root direktori sesi Colab
# print("\n--- Uji Gambar: ngantuk.jpg ---")
# try:
#     image_path_ngantuk = "ngantuk.jpg" # Bisa juga ditulis "/content/ngantuk.jpg"
#     if not os.path.exists(image_path_ngantuk):
#         print(f"File '{image_path_ngantuk}' tidak ditemukan. Harap pastikan nama file dan path sudah benar.")
#     else:
#         test_from_image(image_path_ngantuk, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar ngantuk.jpg: {e}")

# --- Bagian Uji Gambar Lainnya (Dinonaktifkan agar tidak membingungkan) ---
# Contoh 1: Uji gambar mata terbuka (dinonaktifkan)
# print("\n--- Uji Gambar: Mata Terbuka ---")
# try:
#     image_path_open_eyes = "/content/wajah_terbuka.jpg" # GANTI DENGAN PATH GAMBAR ANDA
#     if not os.path.exists(image_path_open_eyes):
#         print(f"File '{image_path_open_eyes}' tidak ditemukan. Harap unggah terlebih dahulu.")
#     else:
#         test_from_image(image_path_open_eyes, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar terbuka: {e}")

# Contoh 2: Uji gambar mata tertutup (dinonaktifkan)
# print("\n--- Uji Gambar: Mata Tertutup ---")
# try:
#     image_path_closed_eyes = "/content/wajah_tertutup.jpg" # GANTI DENGAN PATH GAMBAR ANDA
#     if not os.path.exists(image_path_closed_eyes):
#         print(f"File '{image_path_closed_eyes}' tidak ditemukan. Harap unggah terlebih dahulu.")
#     else:
#         test_from_image(image_path_closed_eyes, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar tertutup: {e}")

# --- Contoh Penggunaan: Uji Video (Aktifkan ini) ---
# Unggah file video Anda ke Colab dan ganti nama file di bawah
print("\n--- Uji Video: ga_ngantuk.mp4 ---")
try:
    video_path_test = "ga_ngantuk.mp4" # Path ke video Anda yang sudah diunggah
    # Anda juga bisa menggunakan "/content/ngantuk.mp4"

    if not os.path.exists(video_path_test):
        print(f"File '{video_path_test}' tidak ditemukan. Harap pastikan nama file dan path sudah benar.")
    else:
        # Untuk video, set display_frames=True jika ingin melihat frame satu per satu (akan menghasilkan banyak output dan mungkin lambat)
        # Atau False untuk hanya melihat ringkasan teks.
        # max_frames bisa dibatasi untuk video panjang agar tidak terlalu lama.
        test_from_video(video_path_test, detector, display_frames=False, max_frames=None) # Set max_frames=None untuk proses semua frame
except Exception as e:
    print(f"Error saat menguji video: {e}")

Detektor siap.

--- Uji Video: ga_ngantuk.mp4 ---
Memproses video: ga_ngantuk.mp4 (80 frames)

--- Ringkasan Video 'ga_ngantuk.mp4' ---
Total frame diproses: 80
Total frame terdeteksi kantuk/menguap: 0
Persentase kantuk: 0.00%


In [ ]:
## Bagian Uji Model

from google.colab import files
import os # Pastikan modul os diimpor untuk os.path.exists

# Inisialisasi Detektor dengan parameter default (bisa Anda ubah di sini)
detector = DrowsinessDetectorStandalone(
    ear_threshold=DEFAULT_EAR_THRESHOLD,
    consecutive_frames_eye=DEFAULT_CONSECUTIVE_FRAMES_EYE,
    mar_threshold=DEFAULT_MAR_THRESHOLD,
    consecutive_frames_yawn=DEFAULT_CONSECUTIVE_FRAMES_YAWN
)

print("Detektor siap.")

# --- Bagian Uji Gambar (Dinonaktifkan agar tidak membingungkan) ---
# Contoh: Uji gambar 'ngantuk.jpg' yang sudah ada di root direktori sesi Colab
# print("\n--- Uji Gambar: ngantuk.jpg ---")
# try:
#     image_path_ngantuk = "ngantuk.jpg" # Bisa juga ditulis "/content/ngantuk.jpg"
#     if not os.path.exists(image_path_ngantuk):
#         print(f"File '{image_path_ngantuk}' tidak ditemukan. Harap pastikan nama file dan path sudah benar.")
#     else:
#         test_from_image(image_path_ngantuk, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar ngantuk.jpg: {e}")

# --- Bagian Uji Gambar Lainnya (Dinonaktifkan agar tidak membingungkan) ---
# Contoh 1: Uji gambar mata terbuka (dinonaktifkan)
# print("\n--- Uji Gambar: Mata Terbuka ---")
# try:
#     image_path_open_eyes = "/content/wajah_terbuka.jpg" # GANTI DENGAN PATH GAMBAR ANDA
#     if not os.path.exists(image_path_open_eyes):
#         print(f"File '{image_path_open_eyes}' tidak ditemukan. Harap unggah terlebih dahulu.")
#     else:
#         test_from_image(image_path_open_eyes, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar terbuka: {e}")

# Contoh 2: Uji gambar mata tertutup (dinonaktifkan)
# print("\n--- Uji Gambar: Mata Tertutup ---")
# try:
#     image_path_closed_eyes = "/content/wajah_tertutup.jpg" # GANTI DENGAN PATH GAMBAR ANDA
#     if not os.path.exists(image_path_closed_eyes):
#         print(f"File '{image_path_closed_eyes}' tidak ditemukan. Harap unggah terlebih dahulu.")
#     else:
#         test_from_image(image_path_closed_eyes, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar tertutup: {e}")

# --- Contoh Penggunaan: Uji Video (Aktifkan ini) ---
# Unggah file video Anda ke Colab dan ganti nama file di bawah
print("\n--- Uji Video: mulut_buka.mp4 ---")
try:
    video_path_test = "Buka_Mulut1.mp4" # Path ke video Anda yang sudah diunggah
    # Anda juga bisa menggunakan "/content/ngantuk.mp4"

    if not os.path.exists(video_path_test):
        print(f"File '{video_path_test}' tidak ditemukan. Harap pastikan nama file dan path sudah benar.")
    else:
        # Untuk video, set display_frames=True jika ingin melihat frame satu per satu (akan menghasilkan banyak output dan mungkin lambat)
        # Atau False untuk hanya melihat ringkasan teks.
        # max_frames bisa dibatasi untuk video panjang agar tidak terlalu lama.
        test_from_video(video_path_test, detector, display_frames=False, max_frames=None) # Set max_frames=None untuk proses semua frame
except Exception as e:
    print(f"Error saat menguji video: {e}")

Detektor siap.

--- Uji Video: mulut_buka.mp4 ---
Memproses video: Buka_Mulut1.mp4 (169 frames)
Peringatan: Alarm kantuk/menguap terpicu! (Suara tidak didukung di Colab)

--- Ringkasan Video 'Buka_Mulut1.mp4' ---
Total frame diproses: 169
Total frame terdeteksi kantuk/menguap: 150
Persentase kantuk: 88.76%


In [ ]:
## Bagian Uji Model

from google.colab import files
import os # Pastikan modul os diimpor untuk os.path.exists

# Inisialisasi Detektor dengan parameter default (bisa Anda ubah di sini)
detector = DrowsinessDetectorStandalone(
    ear_threshold=DEFAULT_EAR_THRESHOLD,
    consecutive_frames_eye=DEFAULT_CONSECUTIVE_FRAMES_EYE,
    mar_threshold=DEFAULT_MAR_THRESHOLD,
    consecutive_frames_yawn=DEFAULT_CONSECUTIVE_FRAMES_YAWN
)

print("Detektor siap.")

# --- Bagian Uji Gambar (Dinonaktifkan agar tidak membingungkan) ---
# Contoh: Uji gambar 'ngantuk.jpg' yang sudah ada di root direktori sesi Colab
# print("\n--- Uji Gambar: ngantuk.jpg ---")
# try:
#     image_path_ngantuk = "ngantuk.jpg" # Bisa juga ditulis "/content/ngantuk.jpg"
#     if not os.path.exists(image_path_ngantuk):
#         print(f"File '{image_path_ngantuk}' tidak ditemukan. Harap pastikan nama file dan path sudah benar.")
#     else:
#         test_from_image(image_path_ngantuk, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar ngantuk.jpg: {e}")

# --- Bagian Uji Gambar Lainnya (Dinonaktifkan agar tidak membingungkan) ---
# Contoh 1: Uji gambar mata terbuka (dinonaktifkan)
# print("\n--- Uji Gambar: Mata Terbuka ---")
# try:
#     image_path_open_eyes = "/content/wajah_terbuka.jpg" # GANTI DENGAN PATH GAMBAR ANDA
#     if not os.path.exists(image_path_open_eyes):
#         print(f"File '{image_path_open_eyes}' tidak ditemukan. Harap unggah terlebih dahulu.")
#     else:
#         test_from_image(image_path_open_eyes, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar terbuka: {e}")

# Contoh 2: Uji gambar mata tertutup (dinonaktifkan)
# print("\n--- Uji Gambar: Mata Tertutup ---")
# try:
#     image_path_closed_eyes = "/content/wajah_tertutup.jpg" # GANTI DENGAN PATH GAMBAR ANDA
#     if not os.path.exists(image_path_closed_eyes):
#         print(f"File '{image_path_closed_eyes}' tidak ditemukan. Harap unggah terlebih dahulu.")
#     else:
#         test_from_image(image_path_closed_eyes, detector)
# except Exception as e:
#     print(f"Error saat menguji gambar tertutup: {e}")

# --- Contoh Penggunaan: Uji Video (Aktifkan ini) ---
# Unggah file video Anda ke Colab dan ganti nama file di bawah
print("\n--- Uji Video: mata_tutup.mp4 ---")
try:
    video_path_test = "mata_tutup.mp4" # Path ke video Anda yang sudah diunggah
    # Anda juga bisa menggunakan "/content/ngantuk.mp4"

    if not os.path.exists(video_path_test):
        print(f"File '{video_path_test}' tidak ditemukan. Harap pastikan nama file dan path sudah benar.")
    else:
        # Untuk video, set display_frames=True jika ingin melihat frame satu per satu (akan menghasilkan banyak output dan mungkin lambat)
        # Atau False untuk hanya melihat ringkasan teks.
        # max_frames bisa dibatasi untuk video panjang agar tidak terlalu lama.
        test_from_video(video_path_test, detector, display_frames=False, max_frames=None) # Set max_frames=None untuk proses semua frame
except Exception as e:
    print(f"Error saat menguji video: {e}")

Detektor siap.

--- Uji Video: mata_tutup.mp4 ---
Memproses video: mata_tutup.mp4 (146 frames)
Peringatan: Alarm kantuk/menguap terpicu! (Suara tidak didukung di Colab)

--- Ringkasan Video 'mata_tutup.mp4' ---
Total frame diproses: 146
Total frame terdeteksi kantuk/menguap: 105
Persentase kantuk: 71.92%
